# Vector Space Model (VSM)
## Using TF-IDF weights from Boolean Model (exercice_1)

In [4]:
import math
import os
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import nltk
import pandas as pd
from collections import defaultdict

# Load data from exercice_1 output files
output_dir = "exercice_1"

# Initialize preprocessing tools (needed for loading)
tokenizer = RegexpTokenizer(
    r'(?:[A-Za-z]\.)+|[A-Za-z]+[\-@]\d+(?:\.\d+)?|\d+(?:[\.\,\-]\d+)*%?|[A-Za-z]+'
)
stop_words = set(stopwords.words('english'))
porter = PorterStemmer()

In [5]:
# Download NLTK resources
nltk.download('stopwords', quiet=True)

# Load documents from Collection folder
collection_path = "../Collection"
documents = {}
for filename in os.listdir(collection_path):
    if filename.endswith(".txt"):
        doc_id = filename.split(".")[0]
        with open(os.path.join(collection_path, filename), "r", encoding="utf-8") as f:
            documents[doc_id] = f.read()

print(f"Loaded {len(documents)} documents")

Loaded 6 documents


In [6]:
# Load TF-IDF weights from exercice_1 output
tfidf_file = os.path.join(output_dir, "TFIDF_Weights.txt")
tfidf_weights = defaultdict(dict)
doc_terms = defaultdict(list)

with open(tfidf_file, "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) >= 3:
            doc_id, term, weight = parts[0], parts[1], float(parts[2])
            tfidf_weights[doc_id][term] = weight
            if term not in doc_terms[doc_id]:
                doc_terms[doc_id].append(term)

# Convert to regular dicts
tfidf_weights = dict(tfidf_weights)
doc_terms = dict(doc_terms)

# Compute IDF from document terms
def compute_idf(doc_terms):
    N = len(doc_terms)
    df = {}
    for terms in doc_terms.values():
        for t in set(terms):
            df[t] = df.get(t, 0) + 1
    return {t: math.log10((N / df[t]) + 1) for t in df}

idf = compute_idf(doc_terms)
vocab = set(idf.keys())

print(f"Vocabulary size: {len(vocab)} terms")
print(f"Number of documents: {len(doc_terms)}")

Vocabulary size: 392 terms
Number of documents: 6


In [7]:
# Text preprocessing function
def preprocess_text(text):
    tokens = tokenizer.tokenize(text)
    tokens = [t.lower() for t in tokens]
    tokens = [porter.stem(t) for t in tokens if t not in stop_words]
    return tokens

In [8]:
# Query vector creation (weighted by IDF)
def create_query_vector(query_text, vocab, idf):
    terms = preprocess_text(query_text)
    unique = set(terms)
    query_vec = {t: idf.get(t, 0) for t in unique if t in vocab}
    return query_vec

In [9]:
# Similarity measures
def inner_product(vd, vq):
    return sum(vd[t] * vq[t] for t in set(vd) & set(vq))

def cosine_similarity(vd, vq):
    num = inner_product(vd, vq)
    den = math.sqrt(sum(w**2 for w in vd.values()) * sum(w**2 for w in vq.values()))
    return 0.0 if den == 0 else num / den

def dice_similarity(vd, vq):
    num = 2 * inner_product(vd, vq)
    den = sum(w**2 for w in vd.values()) + sum(w**2 for w in vq.values())
    return 0.0 if den == 0 else num / den

def jaccard_similarity(vd, vq):
    num = inner_product(vd, vq)
    den = sum(w**2 for w in vd.values()) + sum(w**2 for w in vq.values()) - num
    return 0.0 if den == 0 else num / den

In [10]:
# Document ranking function
def rank_documents(query_vec, doc_vectors, func):
    scores = {d: func(v, query_vec) for d, v in doc_vectors.items()}
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)

In [11]:
# Define queries
queries = {
    "q1": "large language models for information retrieval and ranking",
    "q2": "LLM for information retrieval and Ranking",
    "q3": "query Reformulation in information retrieval",
    "q4": "ranking Documents",
    "q5": "Optimizing recommendation systems with LLMs by leveraging item metadata"
}

# Define similarity measures
measures = {
    "Inner_Product": inner_product,
    "Cosine": cosine_similarity,
    "Dice": dice_similarity,
    "Jaccard": jaccard_similarity
}

In [12]:
# Run all queries and display results as DataFrames
for qn, qt in queries.items():
    print("\n" + "="*70)
    print(f"🔍 {qn}: {qt}")
    
    # Create query vector
    qvec = create_query_vector(qt, vocab, idf)
    print(f"Query terms (stemmed): {sorted(qvec.keys())}")
    print("="*70)
    
    # Collect scores for all similarity measures
    doc_scores = {}
    
    for measure_name, measure_func in measures.items():
        ranked = rank_documents(qvec, tfidf_weights, measure_func)
        for doc, score in ranked:
            if doc not in doc_scores:
                doc_scores[doc] = {}
            doc_scores[doc][measure_name] = score
    
    # Create DataFrame
    df = pd.DataFrame.from_dict(doc_scores, orient='index')
    df.index.name = 'Document'
    
    # Fill missing values with 0
    df = df.fillna(0.0)
    
    # Sort by Inner Product (primary metric)
    df = df.sort_values(by='Inner_Product', ascending=False)
    
    # Filter out documents with all zero scores
    df = df[(df > 0).any(axis=1)]
    
    # Display the table
    print(df.to_string())
    print()


🔍 q1: large language models for information retrieval and ranking
Query terms (stemmed): ['inform', 'languag', 'larg', 'model', 'rank', 'retriev']
          Inner_Product    Cosine      Dice   Jaccard
Document                                             
D2             0.437999  0.217365  0.187574  0.103493
D4             0.276144  0.223399  0.222888  0.125421
D1             0.271529  0.137537  0.119902  0.063774
D5             0.169800  0.080608  0.067969  0.035180
D3             0.133385  0.092312  0.090047  0.047146
D6             0.065353  0.041717  0.039853  0.020332


🔍 q2: LLM for information retrieval and Ranking
Query terms (stemmed): ['inform', 'llm', 'rank', 'retriev']
          Inner_Product    Cosine      Dice   Jaccard
Document                                             
D2             0.479549  0.255711  0.212400  0.118818
D4             0.275143  0.239169  0.236859  0.134339
D1             0.217344  0.118291  0.099366  0.052281
D5             0.196299  0.100129  0.081